# This is a notebook to perform LISCA analysis

Each cell corresponds to a block of code, which can be run by selecting it and pressing shift+enter. The notebook is conceived to be run sequentially. For any errors or questions contact Miguel. Make sure to select the kernel named pyama on the top right. If you cannot find a kernel named pyama, have a look at the confluence page or contact Miguel.

Run the following Cell to make all the imports

In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
from nd2reader import ND2Reader
import pandas as pd
from IPython.display import display
import numpy as np
import sys
import time

sys.path.append('/project/ag-moonraedler/MAtienza/pyama/')
from lisca.pipeline import Track

from tqdm import tqdm
import os
from skimage.io import imread
from notebook_viewer import viewers
%matplotlib ipympl
%load_ext autoreload

Select the folder where your imaging file is located, as well as the folder where you would like for the output of the analysis to be saved

In [2]:
data_path = 'D:/'
nd2_file = 'hek293t-25um-30um-pattern-trafo.nd2'
path_out='D:/'

You can use this tool to have a look at your images and find out which channel (c) is BF and which which channels are fluorscence.

In [ ]:
viewer = viewers.StackViewer(os.path.join(data_path, nd2_file))

In [ ]:
plt.close(viewer.fig)

In [ ]:
first=0
last=24
positions=range(first, last+1)
fl_channel=1
bf_channel=0

In [ ]:
bf_channel=0
fl_channels=[1,2]

If you plan to use cellpose segmentation,use this tool for calibration by checking which parameters work best. Otherwise you may skip this part


In [ ]:
segmentation_viewer = viewers.CellposeViewer(data_path+nd2_file, channel=bf_channel)

In [ ]:
try:
    flow_threshold, diameter, mask_threshold = segmentation_viewer.flow_threshold.value, segmentation_viewer.diameter.value, segmentation_viewer.mask_threshold.value
    plt.close(segmentation_viewer.fig)
except:
    flow_threshold, diameter, mask_threshold = 0.8, 29, -2
    

## Perform LISCA pipeline

In [3]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Chose a method for segmentation from either "th", the standard pyama method or "cellpose"

In [ ]:
method="th"

Choose the positions you want to evalueate

In [ ]:
%autoreload
n_positions=0
positions = range(1)
t_0 = time.time()
for fov in positions:
    print(f'Evaluating field of view {fov}')
    
    fovpath=os.path.join(path_out, f'XY{fov}/')
    if not os.path.isdir(fovpath):
        os.mkdir(fovpath)
    tracker = Track(fovpath, data_path, bf_channel, fl_channels, fov=fov, nd2_file=nd2_file)
    tracker.segment(flow_threshold=flow_threshold, mask_threshold=mask_threshold, diameter=diameter, pretrained_model='mdamb231', method=method)
    tracker.save_to_pyama(fl_channel)
    tracker.track()
    n_positions+=1
print(f"Time taken per position: {(time.time()-t_0)/(60*n_positions)} minutes")

In [26]:
%autoreload
Resultsviewer=viewers.ResultsViewer(data_path+nd2_file, path_out)

D:/XY0/cyto_masks_th.mp4


GridspecLayout(children=(Canvas(header_visible=False, layout=Layout(grid_area='widget001'), toolbar=Toolbar(to…

In [27]:
Resultsviewer.df

,frame,frame.1,x,y,cyto_locator,area,Selected,particle,eGFP_2s,DAPI_NS
0,0,0,846.224490,258.329932,1,294,0,0,154053,351495
1,0,0,1259.552255,1406.517052,39,3636,0,6,1970219,4931657
2,0,0,930.678843,1404.519533,40,1971,0,7,1097945,2720609
3,0,0,1095.415952,1414.925795,41,1981,1,8,1090973,2697989
4,0,0,1099.678033,1555.209785,46,4047,0,12,2151054,5531721
...,...,...,...,...,...,...,...,...,...,...
6314,170,170,93.000000,854.000000,65,45,0,2772,20423,49791
6315,170,170,919.269996,578.932211,41,6889,0,37,72164268,8266021
6316,170,170,1592.673801,568.064017,43,5233,0,38,39556319,6644151
6317,170,170,1270.467087,578.251401,45,2856,1,40,1954735,3357430


In [21]:
Resultsviewer.particle_id

8

In [20]:
Resultsviewer.dfp

,frame,frame.1,x,y,cyto_locator,area,Selected,particle,eGFP_2s,DAPI_NS
5507,152,152,78.748344,1690.980132,109,151,0,3703,66895,176346
5714,157,157,68.000000,1695.000000,112,45,0,3703,19370,51416
5850,160,160,61.240631,1697.745562,126,507,0,3703,222776,580415
5902,161,161,70.510769,1707.049231,116,325,0,3703,142410,374185
5949,162,162,65.409091,1710.893939,136,66,0,3703,28681,75853
5996,163,163,51.647059,1692.598039,130,102,0,3703,43933,115737
6093,165,165,43.878378,1692.148649,123,74,0,3703,31972,83939
6140,166,166,70.039474,1684.118421,129,152,0,3703,67162,175247
6184,167,167,81.057692,1695.692308,124,104,0,3703,45515,120812
6224,168,168,72.500000,1686.000000,117,52,0,3703,23205,59894
